<a href="https://colab.research.google.com/github/linqus/ClassifySpaceRocks/blob/master/notebooks/unit8/nlp_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from distutils.dir_util import copy_tree
from huggingface_hub import Repository, snapshot_download, create_repo, get_full_repo_name


def copy_repository_template():
    # Clone the repo and extract the local path
    template_repo_id = "lewtun/distilbert-base-uncased-finetuned-squad-d5716d28"
    commit_hash = "be3eaffc28669d7932492681cd5f3e8905e358b4"
    template_repo_dir = snapshot_download(template_repo_id, revision=commit_hash)
    # Create an empty repo on the Hub
    model_name = template_repo_id.split("/")[1]
    create_repo(model_name, exist_ok=True)
    # Clone the empty repo
    new_repo_id = get_full_repo_name(model_name)
    new_repo_dir = model_name
    repo = Repository(local_dir=new_repo_dir, clone_from=new_repo_id)
    # Copy files
    copy_tree(template_repo_dir, new_repo_dir)
    # Push to Hub
    repo.push_to_hub()

In [3]:
copy_repository_template()

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/345 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

training_args.bin:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/linqus/distilbert-base-uncased-finetuned-squad-d5716d28 into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/2.05k [00:00<?, ?B/s]

To https://huggingface.co/linqus/distilbert-base-uncased-finetuned-squad-d5716d28
   c663b70..d209bc0  main -> main

   c663b70..d209bc0  main -> main



In [9]:
from transformers import pipeline

model_checkpoint = get_full_repo_name("distilbert-base-uncased-finetuned-squad-d5716d28")
reader = pipeline("question-answering", model=model_checkpoint)


config.json:   0%|          | 0.00/524 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
from huggingface_hub import list_repo_files

list_repo_files(repo_id=model_checkpoint)

['.gitattributes',
 'README.md',
 'pytorch_model.bin',
 'special_tokens_map.json',
 'tokenizer_config.json',
 'training_args.bin',
 'vocab.txt']

In [7]:
from transformers import AutoConfig

pretrained_checkpoint = "distilbert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_checkpoint)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
config.push_to_hub(model_checkpoint, commit_message="Add config.json")

CommitInfo(commit_url='https://huggingface.co/linqus/distilbert-base-uncased-finetuned-squad-d5716d28/commit/399cf0a5e7f2db0e3e6d17da1ba3550f599383c5', commit_message='Add config.json', commit_description='', oid='399cf0a5e7f2db0e3e6d17da1ba3550f599383c5', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
reader = pipeline("question-answering", model=model_checkpoint, revision="main")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a question answering dataset is the SQuAD
dataset, which is entirely based on that task. If you would like to fine-tune a
model on a SQuAD task, you may leverage the
examples/pytorch/question-answering/run_squad.py script.

🤗 Transformers is interoperable with the PyTorch, TensorFlow, and JAX
frameworks, so you can use your favourite tools for a wide variety of tasks!
"""

question = "What is extractive question answering?"
reader(question=question, context=context)

{'score': 0.37610679864883423,
 'start': 34,
 'end': 95,
 'answer': 'the task of extracting an answer from a text\ngiven a question'}



---



In [11]:
tokenizer = reader.tokenizer
model = reader.model

In [12]:
question = "Which frameworks can I use?"

In [16]:
import torch

inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"][0]
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits
# Get the most likely beginning of answer with the argmax of the score
answer_start = torch.argmax(answer_start_scores)
# Get the most likely end of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Which frameworks can I use?
Answer: pytorch, tensorflow, and jax




---



In [17]:
from transformers import AutoTokenizer, AutoModel

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [19]:
text = """
Generation One is a retroactive term for the Transformers characters that
appeared between 1984 and 1993. The Transformers began with the 1980s Japanese
toy lines Micro Change and Diaclone. They presented robots able to transform
into everyday vehicles, electronic items or weapons. Hasbro bought the Micro
Change and Diaclone toys, and partnered with Takara. Marvel Comics was hired by
Hasbro to create the backstory; editor-in-chief Jim Shooter wrote an overall
story, and gave the task of creating the characthers to writer Dennis O'Neil.
Unhappy with O'Neil's work (although O'Neil created the name "Optimus Prime"),
Shooter chose Bob Budiansky to create the characters.

The Transformers mecha were largely designed by Shōji Kawamori, the creator of
the Japanese mecha anime franchise Macross (which was adapted into the Robotech
franchise in North America). Kawamori came up with the idea of transforming
mechs while working on the Diaclone and Macross franchises in the early 1980s
(such as the VF-1 Valkyrie in Macross and Robotech), with his Diaclone mechs
later providing the basis for Transformers.

The primary concept of Generation One is that the heroic Optimus Prime, the
villainous Megatron, and their finest soldiers crash land on pre-historic Earth
in the Ark and the Nemesis before awakening in 1985, Cybertron hurtling through
the Neutral zone as an effect of the war. The Marvel comic was originally part
of the main Marvel Universe, with appearances from Spider-Man and Nick Fury,
plus some cameos, as well as a visit to the Savage Land.

The Transformers TV series began around the same time. Produced by Sunbow
Productions and Marvel Productions, later Hasbro Productions, from the start it
contradicted Budiansky's backstories. The TV series shows the Autobots looking
for new energy sources, and crash landing as the Decepticons attack. Marvel
interpreted the Autobots as destroying a rogue asteroid approaching Cybertron.
Shockwave is loyal to Megatron in the TV series, keeping Cybertron in a
stalemate during his absence, but in the comic book he attempts to take command
of the Decepticons. The TV series would also differ wildly from the origins
Budiansky had created for the Dinobots, the Decepticon turned Autobot Jetfire
(known as Skyfire on TV), the Constructicons (who combine to form
Devastator),[19][20] and Omega Supreme. The Marvel comic establishes early on
that Prime wields the Creation Matrix, which gives life to machines. In the
second season, the two-part episode The Key to Vector Sigma introduced the
ancient Vector Sigma computer, which served the same original purpose as the
Creation Matrix (giving life to Transformers), and its guardian Alpha Trion.
"""

inputs = tokenizer(text, return_tensors="pt")


In [21]:
logits = model(**inputs).logits

RuntimeError: ignored



---



In [1]:
%%capture
!pip install datasets evaluate accelerate

In [59]:
from datasets import load_dataset

import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

raw_datasets = load_dataset("glue", "mnli")
raw_datasets['validation_matched'] = raw_datasets['validation_matched'].select(range(32))

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

#trainer.train()

In [62]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 32
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9847
    })
})

In [28]:
data_collator = trainer.get_train_dataloader().collate_fn
actual_dataset = trainer._remove_unused_columns( trainer.train_dataset )
batch = data_collator([actual_dataset[i] for i in range(4)])


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [29]:
for batch in trainer.get_train_dataloader():
    break

In [32]:
outputs = model(**batch)
outputs

SequenceClassifierOutput(loss=tensor(1.0931, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0052, -0.0044,  0.1149],
        [-0.0467,  0.0243,  0.0660],
        [-0.0673,  0.0160,  0.1127],
        [-0.0650,  0.0733,  0.0879],
        [-0.0682, -0.0093,  0.1045],
        [-0.0652,  0.0096,  0.1220],
        [-0.0657,  0.0443,  0.0587],
        [-0.0903,  0.0078,  0.1210]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [34]:
loss = outputs.loss
loss

tensor(1.0931, grad_fn=<NllLossBackward0>)

In [35]:
loss.backward()

In [45]:
trainer.create_optimizer()

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2e-05
    maximize: False
    weight_decay: 0.01

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2e-05
    maximize: False
    weight_decay: 0.0
)

In [46]:
trainer.optimizer.step()

In [63]:
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.0959302186965942,
 'eval_accuracy': 0.375,
 'eval_runtime': 8.2987,
 'eval_samples_per_second': 3.856,
 'eval_steps_per_second': 0.482}

In [48]:
for batch in trainer.get_eval_dataloader():
    break

In [51]:
import torch

with torch.no_grad():
    outputs = trainer.model(**batch)

outputs

SequenceClassifierOutput(loss=tensor(1.0845), logits=tensor([[-0.0857,  0.1298,  0.0623],
        [-0.1493,  0.1510,  0.0540],
        [-0.1164,  0.1315,  0.0383],
        [-0.1609,  0.1609,  0.0567],
        [-0.1157,  0.1416,  0.0391],
        [-0.0813,  0.1299,  0.0206],
        [-0.1111,  0.1583,  0.0527],
        [-0.1183,  0.0791,  0.0083]]), hidden_states=None, attentions=None)

In [55]:
predictions = outputs.logits.cpu().numpy()
labels = batch["labels"].cpu().numpy()

compute_metrics((predictions, labels))

{'accuracy': 0.25}

In [56]:
predictions.shape, labels.shape

((8, 3), (8,))

In [54]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

In [64]:
device = 'cpu'

for batch in trainer.get_train_dataloader():
    break

batch = {k: v.to(device) for k, v in batch.items()}
trainer.create_optimizer()

for _ in range(20):
    outputs = trainer.model(**batch)
    loss = outputs.loss
    loss.backward()
    trainer.optimizer.step()
    trainer.optimizer.zero_grad()

In [67]:
with torch.no_grad():
    outputs = trainer.model(**batch)
preds = outputs.logits
labels = batch["labels"]

compute_metrics((preds.cpu().numpy(), labels.cpu().numpy()))

{'accuracy': 1.0}

In [68]:
for batch2 in trainer.get_train_dataloader():
    break

In [78]:
batch3 = next(iter(trainer.get_train_dataloader()))
batch4 = next(next(iter(trainer.get_train_dataloader())))

TypeError: ignored

In [77]:
batch3

{'input_ids': tensor([[  101,  2101,  2009,  2001, 11792,  2011, 12790,  1996,  2307,  1010,
          2776,  9854,  4278,  3620,  2146,  2011,  6036,  3620,  2898,  1006,
          1015,  1010,  3998,  2519,  2011,  4278,  2519,  1007,  1010,  1998,
          2071,  2907,  2019,  4378,  1997,  2531,  1010,  2199,  1012,   102,
          2009,  2071,  2025,  2907,  2039,  2000,  2531,  1010,  2199,  2111,
          2044, 12790,  1996,  2307, 11792,  2009,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1045,  2123,  1005,  1056,  2812,  2000,  2022,  1043, 29521,
          2055,  2115,  5936,  1010,  2021,  2065,  1045,  2020,  2017,  1010,
          1045,  2453,  2022,  2062,  4986,  2055,  1996,  2379,  1011,  2744,
          3446, 13494,  1997,  2023,  1002,  1015,  1012,   102,  1996,  2379,
       

In [70]:
batch2

{'input_ids': tensor([[  101,  2101,  2009,  2001, 11792,  2011, 12790,  1996,  2307,  1010,
          2776,  9854,  4278,  3620,  2146,  2011,  6036,  3620,  2898,  1006,
          1015,  1010,  3998,  2519,  2011,  4278,  2519,  1007,  1010,  1998,
          2071,  2907,  2019,  4378,  1997,  2531,  1010,  2199,  1012,   102,
          2009,  2071,  2025,  2907,  2039,  2000,  2531,  1010,  2199,  2111,
          2044, 12790,  1996,  2307, 11792,  2009,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1045,  2123,  1005,  1056,  2812,  2000,  2022,  1043, 29521,
          2055,  2115,  5936,  1010,  2021,  2065,  1045,  2020,  2017,  1010,
          1045,  2453,  2022,  2062,  4986,  2055,  1996,  2379,  1011,  2744,
          3446, 13494,  1997,  2023,  1002,  1015,  1012,   102,  1996,  2379,
       

In [71]:
batch3

{'input_ids': tensor([[  101,  2101,  2009,  2001, 11792,  2011, 12790,  1996,  2307,  1010,
          2776,  9854,  4278,  3620,  2146,  2011,  6036,  3620,  2898,  1006,
          1015,  1010,  3998,  2519,  2011,  4278,  2519,  1007,  1010,  1998,
          2071,  2907,  2019,  4378,  1997,  2531,  1010,  2199,  1012,   102,
          2009,  2071,  2025,  2907,  2039,  2000,  2531,  1010,  2199,  2111,
          2044, 12790,  1996,  2307, 11792,  2009,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1045,  2123,  1005,  1056,  2812,  2000,  2022,  1043, 29521,
          2055,  2115,  5936,  1010,  2021,  2065,  1045,  2020,  2017,  1010,
          1045,  2453,  2022,  2062,  4986,  2055,  1996,  2379,  1011,  2744,
          3446, 13494,  1997,  2023,  1002,  1015,  1012,   102,  1996,  2379,
       

In [81]:
!transformers-cli env

2023-12-17 21:19:42.467817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 21:19:42.467952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 21:19:42.472562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 21:19:48.927425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.35.2
- Platform: Linux-6.1.58+-x86_64-with-gl